# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 100 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
RANDOM_STATE = 42

In [3]:
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [4]:
data_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [5]:
data_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [6]:
data_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [7]:
data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [9]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Данные выгружены, пропусков не обнаружено, дубликаты удалять не надо, так как значения числовые и могут в реальности повторяться. Перейдем к обучению модели. 

## Обучение и проверка модели

Для начала напищем функцию для разделения датафреймов. 

In [10]:
def data_preprocessing(geo_data):
    
    X = geo_data.drop(['id', 'product'], axis=1)
    y = geo_data['product']


    X_train,  X_test, y_train, y_test = train_test_split(
        X, 
        y, 
        test_size=0.25, 
        random_state=RANDOM_STATE
    )
    
    return X_train,  X_test, y_train, y_test

In [11]:
def model_params(
    X_train, 
    X_test, 
    y_train, 
    y_test, 
    model_params
):
    
    num_columns = X_train.columns.tolist()
    
    data_preprocessor = ColumnTransformer(
        [
            ('num', StandardScaler(), num_columns)
        ]
    )
    
    pipe_final = Pipeline([
        ('preprocessor', data_preprocessor),
        ('regressor', model_params[0]['regressor'][0])
    ])
    
    
    grid_search = GridSearchCV(
        pipe_final,
        model_params,
        n_jobs=-1,  
        cv=5, 
        scoring='neg_root_mean_squared_error',  
    )
    grid_search.fit(X_train, y_train)
    
    return grid_search

<b>модель для 1 региона</b>

In [12]:
X_0_train,  X_0_test, y_0_train, y_0_test = data_preprocessing(data_0)

grid_0 = model_params(
    X_0_train, 
    X_0_test, 
    y_0_train, 
    y_0_test, 
    [{
        'regressor': [LinearRegression()],
    }]
)

print('RMSE     :', grid_0.best_score_*(-1))
print('Параметры:\n', grid_0.best_estimator_)

RMSE     : 37.67200536313506
Параметры:
 Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['f0', 'f1', 'f2'])])),
                ('regressor', LinearRegression())])


In [13]:
predict_0 = grid_0.predict(X_0_test)
print('запас сырья:', predict_0.mean())
print('RMSE:', mean_squared_error(y_0_test, predict_0, squared=False))

запас сырья: 92.39879990657768
RMSE: 37.75660035026169


<b>модель для 2 региона</b>

In [14]:
X_1_train,  X_1_test, y_1_train, y_1_test = data_preprocessing(data_1)

grid_1 = model_params(
    X_1_train, 
    X_1_test, 
    y_1_train, 
    y_1_test, 
    [{
        'regressor': [LinearRegression()],
    }]
)

print('RMSE     :', grid_1.best_score_*(-1))
print('Параметры:\n', grid_1.best_estimator_)

RMSE     : 0.8904559629285671
Параметры:
 Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['f0', 'f1', 'f2'])])),
                ('regressor', LinearRegression())])


In [15]:
predict_1 = grid_1.predict(X_1_test)
print('запас сырья:', predict_1.mean())
print('RMSE:', mean_squared_error(y_1_test, predict_1, squared=False))

запас сырья: 68.7128780391376
RMSE: 0.8902801001028846


<b>модель для 3 региона</b>


In [16]:
X_2_train,  X_2_test, y_2_train, y_2_test = data_preprocessing(data_2)

grid_2 = model_params(
    X_2_train, 
    X_2_test, 
    y_2_train, 
    y_2_test, 
    [{
        'regressor': [LinearRegression()],
    }]
)

print('RMSE     :', grid_2.best_score_*(-1))
print('Параметры:\n', grid_2.best_estimator_)

RMSE     : 40.02698159997838
Параметры:
 Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['f0', 'f1', 'f2'])])),
                ('regressor', LinearRegression())])


In [17]:
predict_2 = grid_2.predict(X_2_test)
print('запас сырья:', predict_2.mean())
print('RMSE:', mean_squared_error(y_2_test, predict_2, squared=False))

запас сырья: 94.77102387765939
RMSE: 40.145872311342174


Второй регион демонстрирует наименьшее среднее значение целевого показателя — 69 , что значительно ниже  по сравнению с другими регионами. Однако этот регион отличается наименьшим разбросом прогнозов за счет более низкого RMSE (0.89), что указывает на высокую предсказуемость.

Третий регион, напротив, показывает наибольшее среднее значение целевого показателя — 95, что делает его наиболее перспективным с точки зрения доходности. В то же время, прогнозы для этого региона имеют самый высокий уровень ошибки (RMSE = 40), что свидетельствует о повышенной неопределённости и риске.

Таким образом, второй регион выглядит как наиболее стабильный, но менее прибыльный, тогда как третий регион выделяется высокой потенциальной доходностью при значительных рисках.

## Подготовка к расчёту прибыли

In [18]:
OBJECTS = 500 
BEST_OBJECTS = 200 
BUDGET = 10000000000 
REVENUE = 450000

In [19]:
unbreak_point = (BUDGET / REVENUE) / BEST_OBJECTS
print(f'Точка безубыточности на одну скважин: {unbreak_point} тыс. баррелей')

Точка безубыточности на одну скважин: 111.11111111111111 тыс. баррелей


In [20]:
mean_volume_0 = predict_0.mean()
diff_0 = mean_volume_0 - unbreak_point
print('\nРегион 1:')
print(f'Средний предсказанный запас сырья: {mean_volume_0:.2f} тыс. баррелей')
print(f'Разность с минимальным объёмом: {diff_0:.2f} тыс. баррелей')

mean_volume_1 = predict_1.mean()
diff_1 = mean_volume_1 - unbreak_point
print('\nРегион 2:')
print(f'Средний предсказанный запас сырья: {mean_volume_1:.2f} тыс. баррелей')
print(f'Разность с минимальным объёмом: {diff_1:.2f} тыс. баррелей')

mean_volume_2 = predict_2.mean()
diff_2 = mean_volume_2 - unbreak_point
print('\nРегион 3:')
print(f'Средний предсказанный запас сырья: {mean_volume_2:.2f} тыс. баррелей')
print(f'Разность с минимальным объёмом: {diff_2:.2f} тыс. баррелей')


Регион 1:
Средний предсказанный запас сырья: 92.40 тыс. баррелей
Разность с минимальным объёмом: -18.71 тыс. баррелей

Регион 2:
Средний предсказанный запас сырья: 68.71 тыс. баррелей
Разность с минимальным объёмом: -42.40 тыс. баррелей

Регион 3:
Средний предсказанный запас сырья: 94.77 тыс. баррелей
Разность с минимальным объёмом: -16.34 тыс. баррелей


Так, во всех регионах средние предсказанные запасы меньше, чем точка безубыточности.

## функция для расчёта прибыли

In [21]:
def calculate_profit(X_test, y_test, model, n_wells=BEST_OBJECTS, revenue_per_unit=REVENUE, budget=BUDGET):
 
    predictions = model.predict(X_test)
    y_test = np.array(y_test)
    top_wells_indices = np.argsort(predictions)[-n_wells:]
    total_volume = y_test[top_wells_indices].sum()
    profit = total_volume * revenue_per_unit - budget
    
    return profit

In [22]:
profit_0 = calculate_profit(X_0_test, y_0_test, grid_0.best_estimator_)
print(f'Регион 1:  прибыль = {profit_0:,.2f} руб')

profit_1 = calculate_profit(X_1_test, y_1_test, grid_1.best_estimator_)
print(f'Регион 2:  прибыль = {profit_1:,.2f} руб')

profit_2 = calculate_profit(X_2_test, y_2_test, grid_2.best_estimator_)
print(f'Регион 3:  прибыль = {profit_2:,.2f} руб')

Регион 1:  прибыль = 3,359,141,114.46 руб
Регион 2:  прибыль = 2,415,086,696.68 руб
Регион 3:  прибыль = 2,598,571,759.37 руб


Итак, самым прибыльным регионом при использовании 200 скважин с максимальными значениями, оказался первый

## Расчёт прибыли и рисков 

In [23]:
def bootstrap_profit_analysis(target_test, predict, n_samples=1000, sample_size=OBJECTS):

    df = pd.DataFrame({
        'product': target_test.values,
        'predict': predict
    })
    
    state = np.random.RandomState(RANDOM_STATE)
    profits = []
    
    for _ in range(n_samples):
        df_sample = df.sample(n=sample_size, replace=True, random_state=state)
        top_wells = df_sample.nlargest(BEST_OBJECTS, 'predict')
        total_volume = top_wells['product'].sum()
        revenue = total_volume * REVENUE
        profit = revenue - BUDGET
        profits.append(profit)
    
    profits = pd.Series(profits)
    mean_profit = profits.mean()
    lower_profit = profits.quantile(0.025)
    upper_profit = profits.quantile(0.975)
    loss_risk = (profits < 0).mean() * 100
    
    print(f'95%-й доверительный интервал: от {lower_profit:,.2f} до {upper_profit:,.2f} руб.')
    print(f'Средняя прибыль: {mean_profit:,.2f} руб.')
    print(f'Риск убытков: {loss_risk:.2f}%')
    
    return mean_profit, (lower_profit, upper_profit), loss_risk


mean_profit_0, conf_interval_0, loss_risk_0 = bootstrap_profit_analysis(y_0_test, predict_0)
print()

mean_profit_1, conf_interval_1, loss_risk_1 = bootstrap_profit_analysis(y_1_test, predict_1)
print()

mean_profit_2, conf_interval_2, loss_risk_2 = bootstrap_profit_analysis(y_2_test, predict_2)
print()

95%-й доверительный интервал: от -110,467,895.33 до 897,460,327.72 руб.
Средняя прибыль: 399,575,478.05 руб.
Риск убытков: 6.00%

95%-й доверительный интервал: от 61,684,479.65 до 845,340,177.51 руб.
Средняя прибыль: 452,048,890.91 руб.
Риск убытков: 1.50%

95%-й доверительный интервал: от -144,766,727.30 до 888,390,403.53 руб.
Средняя прибыль: 375,009,902.92 руб.
Риск убытков: 8.00%



In [24]:
viable_regions = []
if loss_risk_0 < 2.5:
    viable_regions.append(('Регион 0', mean_profit_0))
if loss_risk_1 < 2.5:
    viable_regions.append(('Регион 1', mean_profit_1))
if loss_risk_2 < 2.5:
    viable_regions.append(('Регион 2', mean_profit_2))

if viable_regions:
    best_region, best_profit = max(viable_regions, key=lambda x: x[1])
    print(f'Рекомендуемый регион: {best_region}')
    print(f'Обоснование: {best_region} имеет риск убытков {loss_risk_0 if best_region == "Регион 0" else loss_risk_1 if best_region == "Регион 1" else loss_risk_2:.2f}% (< 2.5%) и максимальную среднюю прибыль {best_profit:,.2f} руб.')

Рекомендуемый регион: Регион 1
Обоснование: Регион 1 имеет риск убытков 1.50% (< 2.5%) и максимальную среднюю прибыль 452,048,890.91 руб.


По итогам Bootstrap-анализа для трёх регионов Регион 1 оказался лучшим для разработки скважин. Он показал среднюю прибыль 452 млн руб., риск убытков всего 1.5%  и положительный доверительный интервал (от 62 до 845 млн руб.), что говорит о надёжной рентабельности. Регионы 0 и 2 с рисками 6% и 8% и интервалами, включающими убытки, не соответствуют требованиям. Выбор Региона 1 обусловлен его низким риском и высокой прибылью, что делает его оптимальным для инвестиций.

## Итоговый вывод

Проект для «ГлавРосГосНефть» показал, где лучше бурить новые скважины. Загрузили данные по трём регионам, обучили линейную регрессию, посчитали запасы и риски. Минимальный объём для безубыточности — 111.11 тыс. баррелей, и хотя средние запасы ниже, выбор 200 лучших скважин решает проблему. Bootstrap-анализ (1000 выборок) дал для Региона 1 прибыль 452 млн руб., риск убытков 1.5% и интервал 62–845 млн руб. Регионы 0 и 2 с рисками 6% и 8% не подходят.

Рекомендация: Бурить в Регионе 1 — он самый рентабельный и надёжный с минимальным риском.